<a href="https://colab.research.google.com/github/toanpt74/COLAB_RD/blob/main/ChatBot_Vi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pyvi
from datetime import datetime
import os
import sys
import chromadb
import torch
from peft import (
    LoraConfig,
    get_peft_model,
    get_peft_model_state_dict,
    prepare_model_for_int8_training,
    set_peft_model_state_dict,
    PeftModel,
    PeftConfig,
    prepare_model_for_kbit_training
)

from transformers import (
    AutoConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig
)
from chromadb.utils import embedding_functions

os.environ["WANDB_MODE"] = "offline"
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
base_model = r'D:\Tuan\ChatGPT\model\Vistral-7B-Chat'
device = "auto"
model_id = r"D:\Tuan\ChatGPT\model\phobert-base-v2"
embedding_fun = embedding_functions.SentenceTransformerEmbeddingFunction(model_name=model_id)


def semantic_search(question, num_chunks_distance=5, num_chunks=5):
    collection = client.get_or_create_collection('common', metadata={"hnsw:space": "cosine"},
                                                 embedding_function=embedding_fun)
    results = collection.query(
        query_texts=question,
        include=['distances', 'documents', 'metadatas'],
        n_results=num_chunks,
    )
    print(results)
    image_path = ""
    metadatas = results["metadatas"][0]
    distance = results['distances'][0][0]
    type = metadatas[0]["type"]
    documents = results["documents"][0]

    text = []
    if distance < 0.4:
        if (type == "8system"):
            for j in range(len(metadatas)):
                if metadatas[j]["type"] == "8system":
                    text.append({"question": documents[j], "anwer": metadatas[j]["content"],
                                 "image_path": metadatas[j]["image_path"]})
            # text = metadatas[0]["content"]
            image_path = metadatas[0]["image_path"]
            print('------------------------------------')
            print(text)
            return type, distance, text, image_path
    ids = []
    index = 0
    for cur_chunk_id in results["ids"][0]:
        type_question = metadatas[index]["type"]
        if type_question == 'common':
            cur_chunk_id_index = int(cur_chunk_id.split('_')[-1])
            head_id = cur_chunk_id[0:len(cur_chunk_id) - len(str(cur_chunk_id_index))]
            for i in range(-num_chunks_distance, num_chunks_distance + 1, 1):
                if cur_chunk_id_index + i >= 0:
                    ids.append(head_id + str(cur_chunk_id_index + i))

        index = index + 1
    ids = sorted(set(ids))
    results = results if len(ids) == 1 else collection.get(ids=ids)
    context = " ".join(s for s in results["documents"])
    return type, distance, context, image_path


bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
)
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16
)

tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True, local_files_only=True)

tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

model = AutoModelForCausalLM.from_pretrained(
    base_model,
    device_map=device,
    quantization_config=quantization_config,
    local_files_only=True
)

system_prompt = "Bạn là một trợ lí Tiếng Việt nhiệt tình và trung thực. Hãy luôn trả lời một cách hữu ích nhất có thể, đồng thời giữ an toàn.\n"
system_prompt += "Câu trả lời của bạn không nên chứa bất kỳ nội dung gây hại, phân biệt chủng tộc, phân biệt giới tính, độc hại, nguy hiểm hoặc bất hợp pháp nào. Hãy đảm bảo rằng các câu trả lời của bạn không có thiên kiến xã hội và mang tính tích cực."
system_prompt += "Nếu một câu hỏi không có ý nghĩa hoặc không hợp lý về mặt thông tin, hãy giải thích tại sao thay vì trả lời một điều gì đó không chính xác. Nếu bạn không biết câu trả lời cho một câu hỏi, hãy trẳ lời là bạn không biết và vui lòng không chia sẻ thông tin sai lệch.\n"

client = chromadb.PersistentClient('QAVectorDB_Vi')
collection = client.get_or_create_collection('common', metadata={"hnsw:space": "cosine"},
                                             embedding_function=embedding_fun)

while True:
    question = input("Nhap cau hoi:")
    if question == "":
        break
    type, distance, context, image_path = semantic_search(question, num_chunks_distance=1)

    if type == "common":
        print(context)
        system_prompt += context
        conversation = [{"role": "system", "content": system_prompt}]
        conversation.append({"role": "user", "content": f"{question}"})

        input_ids = tokenizer.apply_chat_template(conversation, return_tensors="pt").to(model.device)
        print(input_ids[0].shape)

        out_ids = model.generate(
            input_ids=input_ids,
            max_new_tokens=1000,
            do_sample=True,
            top_p=0.95,
            top_k=40,
            temperature=0.1,
            repetition_penalty=1.05,
        )
        assistant = tokenizer.batch_decode(out_ids[:, input_ids.size(1):], skip_special_tokens=True)[0].strip()
        print("Assistant: ", assistant)
        conversation.append({"role": "assistant", "content": assistant})
    elif (type == "8system"):
        print(context)
        print(image_path)

    # inputs_not_chat = tokenizer.encode_plus("[INST] Tell me about fantasy football? [/INST]", return_tensors="pt")['input_ids'].to('cuda')
    #
    # generated_ids = model.generate(inputs_not_chat,
    #                                max_new_tokens=1000,
    #                                do_sample=True)
    # decoded = tokenizer.batch_decode(generated_ids)
